# n-gram 모델
- n-gram 모델은 자연어 처리, 정보 검색 등에서 활용이 되는 시퀀스 데이터 표현 방식입니다. 
- ngram에서 n은 연속된 단어의 개수를 의미합니다. 

* n=1 : 1-gram(unigram)
* n=2 : 2-gram(bigram)
* n=3 : 3-gram(trigram)
* n=4 : 4-gram

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt

In [ ]:
# 어절 단위 n-gram
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text))]
    return tuple(ngrams)


# 음절 n-gram 분석
def phoneme_ngram(bow, num_gram):
    sentence = ' '.join(bow)
    text = tuple(sentence)
    slen = len(text)
    ngrams = [text[x:x + num_gram] for x in range(0, slen)]
    return ngrams


# 유사도 계산
def similarity(doc1, doc2):
    cnt = 0
    for token in doc1:
        if token in doc2:
            cnt = cnt + 1

    return cnt/len(doc1)


In [ ]:
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다'
sentence3 = '나는 맛잇는 밥을 뉴턴 선생님과 함께 먹었습니다.'

In [ ]:
okt = Okt()
bow1 = okt.nouns(sentence1)
bow2 = okt.nouns(sentence2)
bow3 = okt.nouns(sentence3)

doc1 = word_ngram(bow1, 2)
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

print(doc1)
print(doc2)
print(doc3)

r1 = similarity(doc1, doc2)
r2 = similarity(doc3, doc1)
print(r1)
print(r2)

(('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
(('나', '맛'), ('맛', '밥'), ('밥', '뉴턴'), ('뉴턴', '선생님'), ('선생님',))
0.6
0.2


# 코사인 유사도

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
# 코사인 유사도 계산
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))


# TDM 만들기
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}

    for word in word_dics:
        freq_mat[word] = 0

    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1

    return freq_mat


# 단어 벡터 만들기
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    return vec

In [ ]:
# 문장 정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다'
sentence3 = '나는 맛잇는 밥을 뉴턴 선생님과 함께 먹었습니다.'

# 헝태소분석기를 이용해 단어 묶음 리스트 생성
okt = Okt()
bow1 = okt.nouns(sentence1)
bow2 = okt.nouns(sentence2)
bow3 = okt.nouns(sentence3)

# 단어 묶음 리스트를 하나로 합침
bow = bow1 + bow2 + bow3

# 단어 묶음에서 중복제거해 단어 사전 구축
word_dics = []
for token in bow:
    if token not in word_dics:
        word_dics.append(token)


# 문장 별 단어 문서 행렬 계산
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)
freq_list3 = make_term_doc_mat(bow3, word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)


# 코사인 유사도 계산
doc1 = np.array(make_vector(freq_list1))
doc2 = np.array(make_vector(freq_list2))
doc3 = np.array(make_vector(freq_list3))
r1 = cos_sim(doc1, doc2)
r2 = cos_sim(doc3, doc1)
print(r1)
print(r2)


{'뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '나': 0, '맛': 0, '밥': 0}
{'뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '나': 0, '맛': 0, '밥': 0}
{'뉴턴': 1, '선생님': 1, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '나': 1, '맛': 1, '밥': 1}
0.7999999999999998
0.3999999999999999


https://soyoung-new-challenge.tistory.com/34

In [ ]:
sentence = ( "휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.",
	     "폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니다.")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

# 문장 벡터화
tfidf_matrix = tfidf_vectorizer.fit_transform(sentence)

# 각 단어
text = tfidf_vectorizer.get_feature_names()

# 각 단어의 벡터 값
idf = tfidf_vectorizer.idf_

In [ ]:
print(dict(zip(text, idf)))

{'갑작스런': 1.4054651081081644, '내일': 1.4054651081081644, '놀러왔다가': 1.4054651081081644, '망연자실': 1.4054651081081644, '반가운': 1.4054651081081644, '서쪽': 1.4054651081081644, '소식': 1.4054651081081644, '오늘': 1.4054651081081644, '으로': 1.4054651081081644, '이어졌는데요': 1.4054651081081644, '인해': 1.4054651081081644, '있습니다': 1.0, '중심': 1.4054651081081644, '폭염': 1.0, '피해서': 1.4054651081081644, '하고': 1.4054651081081644, '휴일': 1.0}


## 자카드 유사도( Jaccard Similarity)
- 두 문장을 각각 단어의 집합으로 만든 뒤 두 집합을 통해 유사도를 측정하는 방식 중 하나
- 유사도를 측정하는 방법은 두 집합의 교집합인 공통된 단어의 개수를 두 집합의 합집합, 전체 단어의 갯수로 나눈다.
- 결과값은 공통의 원소의 개수에 따라 0과 1사이의 값이 나올 것이고, 1에 가까울수록 유사도가 높다

## 코사인 유사도

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

array([[0.17952266]])

## 유클리디언 유사도 ( L2 Distance )

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2])

array([[1.28099753]])

### 벡터를 일반화 + 유클리디언 유사도 
- 앞서 확인했던 유사도 방식들은 모두 0과 1사이의 값을 가졌는데, 유클리디언 유사도는 1보다 큰 값이 나왔다.

- 유클리디언 유사도는 단순히 두 점 사이의 거리를 뜻하므로 값에 제한이 없다.

- 다른 유사도와 비교하려면 0과 1사이의 값으로 맞춰주어야한다.

- 앞서 구한 벡터를 일반화한 후 다시 유클리디언 유사도를 측정하면 0과 1사이의 값을 가진다.

In [ ]:
import numpy as np

def l1_normalize(v):
    norm = np.sum(v)
    return v / norm
    
    
tfidf_norm_l1 = l1_normalize(tfidf_matrix)

euclidean_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.20491229]])

## 맨하탄 유사도 ( L1 Distance )

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import manhattan_distances

def l1_normalize(v):
    norm = np.sum(v)
    return v / norm 
  
# L1 정규화  
tfidf_norm_l1 = l1_normalize(tfidf_matrix)

manhattan_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.77865927]])

## 맨하탄 유사도로 측정했을 때 유사도가 가장 높게 나왔다. 측정 방법에 따라 크게 유사도가 달라질 수 있으므로, 의도하고자 하는 방향에 맞는 유사도측정 방법을 고르는 것이 중요하다

In [ ]:
## - 맨하탄 유사도로 측정했을 때 유사도가 가장 높게 나왔다. 측정 방법에 따라 크게 유사도가 달라질 수 있으므로, 의도하고자 하는 방향에 맞는 유사도측정 방법을 고르는 것이 중요하다